#*Numpy часть*

Выполняется с помощью numpy и базовой функциональности python.

    1. Получить датасет Ирис

    2. Представить датасет одномерный/двумерный массивы (признаки
    поместить в матрицу (n, 4), названия самого ириса --- вектор
    размера n).

    3. Отобразите сколько места он занимает

    4. Нормализуйте данные на промежуток от 0 до 1 для каждой колонки

    5. Представьте один из признаков в виде категориальной переменной
    её на три типа используя квантили:
        < 0.25 --- small
        0.25-0.75 --- medium
        > 0.75 --- big
        
    6. Разделите датасет на две случайные выборки (0.8 / 0.2)


In [29]:
import numpy as np

ira = np.genfromtxt('/content/drive/MyDrive/Iris.csv',
                    delimiter=',',
                    dtype=None,
                    names=True,
                    encoding='utf-8')
iris = ira
for i in range(3):
    print(iris[i])

(1, 5.1, 3.5, 1.4, 0.2, 'Iris-setosa')
(2, 4.9, 3.0, 1.4, 0.2, 'Iris-setosa')
(3, 4.7, 3.2, 1.3, 0.2, 'Iris-setosa')


In [30]:
matrix = []
for i in range(len(iris)):
    victor = iris[i]
    name = victor[-1]
    victOr = list(victor)[1:-1]
    matrix_str = np.concatenate([[name], victOr])
    matrix.append(matrix_str)

matrix2 = np.array(matrix)
print(matrix2[:5])

[['Iris-setosa' '5.1' '3.5' '1.4' '0.2']
 ['Iris-setosa' '4.9' '3.0' '1.4' '0.2']
 ['Iris-setosa' '4.7' '3.2' '1.3' '0.2']
 ['Iris-setosa' '4.6' '3.1' '1.5' '0.2']
 ['Iris-setosa' '5.0' '3.6' '1.4' '0.2']]


In [31]:
def mem_use(arr):
    all_bytes = arr.nbytes
    if arr.dtype.names is not None:
        for name in arr.dtype.names:
            if arr.dtype[name].kind in ['U', 'S']:
                for item in arr[name]:
                    if item is not None:
                        all_bytes += len(item.encode('utf-8'))
    return all_bytes / 1024

mermaid_memory = mem_use(iris)
print(mermaid_memory)

16.6015625


In [32]:
data = np.column_stack([iris['SepalLengthCm'],
                       iris['SepalWidthCm'],
                       iris['PetalLengthCm'],
                       iris['PetalWidthCm']])

def min_max_normalize(column):
    min_val = np.min(column)
    max_val = np.max(column)
    return (column - min_val) / (max_val - min_val)

normalized = np.apply_along_axis(min_max_normalize, axis=0, arr=data)
print(normalized[:5])

[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]


In [33]:
length = iris['PetalLengthCm']
quanti = np.quantile(length, [0, 0.25, 0.75, 1])
labels = ['small', 'medium', 'big']

id = np.searchsorted(quanti, length, side='right') - 1
id = np.clip(id, 0, 2)
dota = np.array(labels)[id]

unique, counts = np.unique(dota, return_counts=True)
for label, count in zip(unique, counts):
    print(f"{label}: {count}")

big: 42
medium: 71
small: 37


In [34]:
np.random.seed(42)
shuffy = np.random.permutation(len(iris))
dota2 = iris[shuffy]

train_size = int(0.8 * len(iris))
train_data = dota2[:train_size]
val_data = dota2[train_size:]

print(len(train_data))
print(len(val_data))

120
30


#Выполняется с помощью пакетов sklearn и numpy

    1. Выберите один из методов классификации (к примеру, SVC), обучите на тренировочном датасете, оцените результат
    на тестовом и сделайте выводы

    3. Проведите три эксперимента изменяя гиперпараметры модели или
    условия препроцессинга данных (например, нормализованные или нет данные), сделайте выводы

    4. Пользуясь методами уменьшения размерности (PCA, t-sne)
    визуализируйте датасет. Это должен быть scatter plot, где точка
    это отдельный ирис, ее координаты -- признаки со сжатой выбранной
    вам размерностью, а цвет -- тип ириса. Нарисуйте этот график дважды, где:

        1 - Тип ириса выбирается из оригинальных данных
        2 - Тип ириса выбирается из предсказанных данных


In [35]:
from sklearn import svm

x_train_data = []
y_train_data = []
x_val_data = []
y_val_data = []

for i in range(len(train_data)):
    vect = train_data[i]
    num_feat = [vect[1], vect[2], vect[3], vect[4]]
    name = vect[-1]
    x_train_data.append(num_feat + [1])
    y_train_data.append(name)

for i in range(len(val_data)):
    vect = val_data[i]
    numeric_features = [vect[1], vect[2], vect[3], vect[4]]
    name = vect[-1]
    x_val_data.append(numeric_features + [1])
    y_val_data.append(name)

if len(x_train_data) > 0 and len(x_val_data) > 0:
    ker = svm.SVC(kernel='rbf')
    ker.fit(x_train_data, y_train_data)
    pred = ker.predict(x_val_data)

    print("Predictions", pred)
    print("Truth", y_val_data)

    correct_predictions = np.sum(np.array(pred) == np.array(y_val_data))
    accuracy = (correct_predictions / len(pred)) * 100

    print(f"Correct predictions - {correct_predictions}, from {len(pred)}")
    print(f"Procentage - {accuracy}")

    sup = ker.support_vectors_
    print("Support vectors", sup.shape)

Predictions ['Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica']
Truth [np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virgini

Вывод: Модель успешно предсказывает 29 из 30 правильных видов = 96.67%

In [36]:
ker1 = svm.SVC(kernel='linear')
ker1.fit(x_train_data, y_train_data)
pred = ker1.predict(x_val_data)
print("Linear kernel predictions", pred)
print("Truth", y_val_data)

correct = np.sum(np.array(pred) == np.array(y_val_data))
accuracy = (correct / len(pred)) * 100
print(f"Linear accuracy: {accuracy}% ({correct}/{len(pred)} is correct)")

sup1 = ker1.support_vectors_
print("Support vectors", sup1.shape)

Linear kernel predictions ['Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica']
Truth [np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_

Вывод: смена ядра с нелинейного на линейное сокращает матрицу опорных векторов, но по сути ее значения не изменяются, поэтому не изменяется и предсказание остается прежним

In [37]:
ker2 = svm.SVC(kernel='poly')
ker2.fit(x_train_data, y_train_data)
pred = ker2.predict(x_val_data)
print("Poly kernel predictions", pred)
print("Truth", y_val_data)

correct0 = np.sum(np.array(pred) == np.array(y_val_data))
accuracy0 = (correct0 / len(pred)) * 100
print(f"Poly kernel accuracy {accuracy0}% ({correct0}/{len(pred)} is correct)")

sup2 = ker2.support_vectors_
print("support vectors", sup2.shape)

Poly kernel predictions ['Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica']
Truth [np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('

Вывод: смена ядра с нелинейного на полиномиальное сокращает матрицу опорных векторов, но по сути ее значения не изменяются, поэтому не изменяется и предсказание

In [38]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

x_train_data_a = np.array([x[:-1] for x in x_train_data])
x_test_data_a = np.array(x_val_data)[:, :-1]

scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = scaler.fit_transform(x_train_data_a)
x_test_scaled = scaler.transform(x_test_data_a)

ker = svm.SVC(kernel='linear')
ker.fit(x_train_scaled, y_train_data)
pred = ker.predict(x_test_scaled)
print("Scaled linear predictions", pred)
print("True validation labels", y_val_data)

correct1 = np.sum(np.array(pred) == np.array(y_val_data))
accuracy = (correct1 / len(pred)) * 100
print(f"Scaled linear accuracy {accuracy}% ({correct1}/{len(pred)} correct)")

sup = ker.support_vectors_
print("Scaled support vectors shape", sup.shape)

Scaled linear predictions ['Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica']
True validation labels [np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-versicolor'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-virginica'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-virginica'), np.str_('Iris-setosa'), np.str_('Iris-versicolor'), np.str_('Iris-vir

Нормализация данных при помощи MinMaxScaler к [0, 1] меняет предсказание. Его точность не изменяется, хотя количество опрных векторов меняется.


In [40]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA

all_data = x_train_scaled.tolist() + x_test_scaled.tolist()
all_labels = y_train_data + y_val_data
all_pred = list(ker.predict(x_train_scaled)) + list(pred)

pcashka = PCA(n_components=3)
my_pca = pcashka.fit_transform(all_data)

train_size = len(x_train_scaled)
train_pca = my_pca[:train_size]
test_pca = my_pca[train_size:]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Reality: circle - train, square - test', 'Prediction: circle - train, square - test')
)

colors = {'Iris-setosa': 'red', 'Iris-versicolor': 'green', 'Iris-virginica': 'blue'}
markers = {'train': 'circle', 'test': 'square'}

for species in colors:
    mask_train = [label == species for label in y_train_data]
    if any(mask_train):
        fig.add_trace(go.Scatter(
            x=train_pca[mask_train, 0], y=train_pca[mask_train, 1],
            mode='markers',
            marker=dict(color=colors[species], size=10, symbol='circle'),
            name=f'{species} (train)',
            legendgroup=species), row=1, col=1)

    mask_test = [label == species for label in y_val_data]
    if any(mask_test):
        fig.add_trace(go.Scatter(
            x=test_pca[mask_test, 0], y=test_pca[mask_test, 1],
            mode = 'markers',
            marker=dict(color=colors[species], size=12, symbol='square', line=dict(width=2, color='black')),
            name=f'{species} (test)',
            legendgroup=species,
            showlegend=False), row=1, col=1)

for species in colors:
    mask_train_pred = [pred == species for pred in all_pred[:train_size]]
    if any(mask_train_pred):
        fig.add_trace(go.Scatter(
            x=train_pca[mask_train_pred, 0], y=train_pca[mask_train_pred, 1],
            mode='markers',
            marker=dict(color=colors[species], size=10, symbol='circle'),
            name=f'{species} (train)',
            legendgroup=species,
            showlegend=False), row=1, col=2)

    mask_test_pred = [pred == species for pred in all_pred[train_size:]]
    if any(mask_test_pred):
        fig.add_trace(go.Scatter(
            x=test_pca[mask_test_pred, 0], y=test_pca[mask_test_pred, 1],
            mode='markers',
            marker=dict(color=colors[species], size=12, symbol='square', line=dict(width=2, color='black')),
            name=f'{species} (test)',
            legendgroup=species,
            showlegend=False), row=1, col=2)

fig.update_xaxes(title_text="PC1", row=1, col=1)
fig.update_yaxes(title_text="PC2", row=1, col=1)
fig.update_xaxes(title_text="PC1", row=1, col=2)
fig.update_yaxes(title_text="PC2", row=1, col=2)

fig.show()